In [14]:
import pandas as pd
import geopandas as gpd
import numpy as np
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import math

In [15]:
# df_cem = pd.read_excel('data/CEM/Tabelas_MCidades.xls')

In [16]:
# df_cem

In [17]:
dimensoes = [
    # 'situacao_setor',#
    # 'tipo_setor',#
    'numero_de_domicilios',#
    'numero_moradores',#
    'porcentagem_domicilios_tipo_comodo', #
    'porcentagem_domicilios_sem_coleta_de_lixo', #
    'porcentagem_domicilios_sem_ligacao_rede_de_agua', #
    'porcentagem_domicilios_sem_banheiros', #
    'porcentagem_domicilios_sem_esgoto_ou_fossa', #
    'porcentagem_domicilios_cedidos_outra_forma', #
    'numero_de_banheiros_por_habitante', #
    'porcentagem_responsaveis_nao_alfabetizado', #
    'porcentagem_responsaveis_nao_alfabetizado_com_menos_de_30_anos', #
    'porcentagem_de_responsaveis_com_menos_30_anos', #
    'porcentagem_responsaveis_renda_ate_3_salarios', #
    'porcentagem_responsaveis_menos_8_anos_de_estudo', #
    'anos_medios_estudo_responsavel', #
    'renda_media_responsavel', #
    'numero_de_domicilios_improvisados', #
    'numero_medio_pessoas', #
]

In [18]:
df = pd.read_csv('resultados/modelo-2000.csv.zip')
df.fillna(0, inplace=True)
# rmsp = (df.rm == 20)
subnormal = (df.tipo_setor == 1)
df.loc[:, 'subnormal'] = 0
df.loc[subnormal, 'subnormal'] = 1
df.loc[:, 'processado'] = False

In [19]:
## TODO
# Exclui setores com menos de 50 domicílios
df[df.numero_de_domicilios < 50]

,Cod_setor,cod_grande_regiao,uf,nome_UF,rm,nome_rm,meso_regiao,cod_municipio,nome_municipio,situacao_setor,...,porcentagem_responsaveis_nao_alfabetizado_com_menos_de_30_anos,porcentagem_de_responsaveis_com_menos_30_anos,porcentagem_responsaveis_renda_ate_3_salarios,porcentagem_responsaveis_menos_8_anos_de_estudo,anos_medios_estudo_responsavel,renda_media_responsavel,numero_de_domicilios_improvisados,numero_medio_pessoas,subnormal,processado
3,120001305000004,1.0,12,AC,0,Não pertence,1202,1200013,Acrelândia,5,...,0.142857,0.142857,0.755102,0.510204,168.0,383.122449,0,4.183673,0,False
12,120005405000004,1.0,12,AC,0,Não pertence,1202,1200054,Assis Brasil,8,...,0.281250,0.343750,0.375000,0.562500,0.0,90.250000,0,4.468750,0,False
13,120005405000005,1.0,12,AC,0,Não pertence,1202,1200054,Assis Brasil,8,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0,0.000000,0,False
14,120005405000006,1.0,12,AC,0,Não pertence,1202,1200054,Assis Brasil,8,...,0.074074,0.296296,0.888889,0.111111,0.0,141.592593,0,6.740741,0,False
37,120017905000002,1.0,12,AC,0,Não pertence,1202,1200179,Capixaba,8,...,0.100000,0.233333,0.833333,0.500000,0.0,291.866667,0,3.733333,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215779,172120805000024,1.0,17,TO,0,Não pertence,1701,1721208,Tocantinópolis,7,...,0.090909,0.227273,0.272727,0.227273,0.0,54.863636,0,8.590909,0,False
215805,172210705000010,1.0,17,TO,0,Não pertence,1701,1722107,Xambioá,5,...,0.090909,0.090909,0.848485,0.484848,49.0,184.848485,0,4.727273,0,False
215806,172210705000011,1.0,17,TO,0,Não pertence,1701,1722107,Xambioá,5,...,0.071429,0.107143,0.857143,0.464286,36.0,185.178571,0,4.107143,0,False
215809,172210705000014,1.0,17,TO,0,Não pertence,1701,1722107,Xambioá,5,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0,3.500000,0,False


In [20]:
df.tipo_setor.value_counts()

tipo_setor
0    206570
1      7871
7       578
5       387
6       219
2        96
3        87
4         3
Name: count, dtype: int64

In [21]:
r_subnormal = df.loc[(subnormal), ['rm', 'subnormal']].groupby('rm').count().reset_index()
cidades_fora_rms = r_subnormal[(r_subnormal.subnormal < 20) | (r_subnormal.rm == 0.)].rm.to_list()
rms = r_subnormal[(r_subnormal.subnormal >= 20) & (r_subnormal.rm != 0.)].rm.to_list()

In [22]:
rms

[1, 2, 3, 5, 6, 7, 8, 10, 13, 14, 15, 16, 17, 26, 27, 28]

In [23]:
df.loc[:, ['rm', 'nome_rm']].groupby('rm').first()

,nome_rm
rm,
0,Não pertence
1,Belém
2,Grande São Luís
3,Fortaleza
4,Natal
5,Recife
6,Maceió
7,Salvador
8,Belo Horizonte


In [39]:
def processa_modelos(filter, rm=None, uf=None):

    registro = {}

    # filter = (df.rm == rm)
    df_rm = df.loc[(filter)]
    df.loc[(filter), 'processado'] = True
    print(df.loc[(filter)].shape)

    # print(df_rm.nome_rm.unique()[0])
    if rm:
        registro['nome_rm_ou_UF'] = df_rm.nome_rm.unique()[0]
    
    if uf:
        registro['nome_rm_ou_UF'] = uf
    # print(df_rm.subnormal.value_counts())

    registro['quantidade_de_municipios_processados'] = df_rm.cod_municipio.unique().shape[0]

    setores_normais = df_rm.loc[filter & (df_rm.loc[filter].subnormal == 0)].shape[0]
    registro['quantidade_original_de_setores_normais'] = setores_normais
    
    setores_subnormais = df_rm.loc[filter & (df_rm.loc[filter].subnormal == 1)].shape[0]
    registro['quantidade_original_de_setores_subnormais'] = setores_subnormais
    # print(setores_normais, setores_subnormais)
    

    samples = min(setores_normais, math.floor(setores_subnormais * 2))
    # samples = 1

    # prepara amostras e lições
    x = df_rm.loc[(filter) & (subnormal)]
    x_ = df_rm.loc[(filter) & (~subnormal)].sample(n=samples, random_state=13)
    X = pd.concat([x, x_]).loc[(filter), dimensoes]
    transformer = Normalizer().fit(X)
    
    # X = transformer.transform(X)

    y = pd.concat([x, x_]).loc[(filter), 'subnormal']
    # X = df_rm.loc[(filter), dimensoes]
    # y = df_rm.loc[(filter), 'subnormal']

    # Processa os modelos, LDA
    # print(y.value_counts())
    clf = LinearDiscriminantAnalysis(solver='svd')
    clf.fit(X, y)
    print(clf.coef_)
    # print(np.unique(clf.predict(X), return_counts=True))
    # predicao = clf.predict(transformer.transform(df_rm.loc[(filter), dimensoes]))
    predicao = clf.predict(df_rm.loc[(filter), dimensoes])
    try: 
        atribuidos_como_subnormais = np.unique(predicao, return_counts=True)[1][1]
    except:
        atribuidos_como_subnormais = 0
    registro['atribuidos_como_subnormais_LDA'] = atribuidos_como_subnormais
    # print(clf.score(X, y))
    registro['proporcao_atribuido_original_LDA'] = atribuidos_como_subnormais / setores_subnormais
    registro['score_LDA'] = clf.score(X, y)
    df.loc[(filter), 'subnormal_LDA'] = predicao

    # # Processa model RandonForest
    # clf = RandomForestClassifier(max_depth=10, random_state=0)
    # clf.fit(X, y)
    # # print(np.unique(clf.predict(X), return_counts=True))
    # # predicao = clf.predict(transformer.transform(df_rm.loc[(filter), dimensoes]))
    # predicao = clf.predict(df_rm.loc[(filter), dimensoes])
    # try:
    #     atribuidos_como_subnormais = np.unique(predicao, return_counts=True)[1][1]
    # except:
    #     atribuidos_como_subnormais = 0    
    # registro['atribuidos_como_subnormais_RandomForest'] = atribuidos_como_subnormais
    # # print(clf.score(X, y))
    # registro['proporcao_atribuido_original_RandomForest'] = atribuidos_como_subnormais / setores_subnormais
    # registro['score_RandomForest'] = clf.score(X, y)
    # df.loc[(filter), 'subnormal_RandomForest'] = predicao


    # # Processa modelo SVC
    # clf = LinearSVC(random_state=0, tol=1e-5)
    # clf.fit(Normalizer().fit_transform(X), y)
    # # print(np.unique(clf.predict(X), return_counts=True))
    # predicao = clf.predict(Normalizer().fit_transform(df_rm.loc[(filter), dimensoes]))
    # try:
    #     atribuidos_como_subnormais = np.unique(clf.predict(Normalizer().fit_transform(df_rm.loc[(filter), dimensoes])), return_counts=True)[1][1]
    # except:
    #     atribuidos_como_subnormais = 0    
    # registro['atribuidos_como_subnormais_SVC'] = atribuidos_como_subnormais
    # # print(clf.score(X, y))
    # registro['proporcao_atribuido_original_SVC'] = atribuidos_como_subnormais / setores_subnormais
    # registro['score_SVC'] = clf.score(Normalizer().fit_transform(X), y)
    # df.loc[(filter), 'subnormal_SVC'] = predicao

    # Salva as predicoes em atributos
    return registro

In [40]:
# Processa cada um dos municípios da sua Região MEtropolitana
result = [] 

for rm in rms:

    rm_filter = (df.rm == rm)
    registro = processa_modelos(rm_filter, rm=rm)
    # Salva as predicoes em atributos
    result.append(registro)

(1666, 32)
[[-1.55105148e-02  3.62972967e-03 -1.83715046e-01 -2.06283816e+00
  -5.99641873e-01  1.65042967e+00  2.68326369e+00 -1.49675395e+00
  -1.36240810e+00  7.39335567e-01  3.42962597e+01 -2.96738072e+01
   2.26236942e-01  3.73103508e+00 -3.52400889e-04  6.65392982e-04
  -2.80740215e-02 -7.19679411e-02]]
(985, 32)
[[ 4.90218813e-03 -1.24170352e-03  3.49774506e+00  2.31202990e+00
   8.80372750e-01 -2.39307431e+00  8.32627134e-01  1.03354365e+01
  -1.15015243e+00 -4.33267786e+00  6.35573081e+01 -6.43088372e+01
   7.31149497e-01 -2.47234052e+00  2.97730767e-04  5.64586909e-04
   2.32196519e-02  4.23230692e-02]]
(2987, 32)
[[ 4.09009238e-03 -1.72849301e-03  9.18955709e+00 -3.15714242e+00
  -2.02518613e+00  2.03280100e+00 -6.97962168e-01 -3.21178277e+00
  -2.35368321e-01 -6.71743359e+00  9.03114445e+00 -5.43931265e+00
   1.77922454e-01  4.47718797e+00 -4.98424133e-04  4.98745350e-04
  -1.23620331e-02  9.27361021e-01]]
(2806, 32)
[[ 1.36411745e-02 -2.83561287e-03  4.06311520e+00 -3.4772

In [26]:
clf.coef_

NameError: name 'clf' is not defined

In [12]:
df.loc[df.rm.isin([13, 14, 15, 16]) & (df.subnormal == 0) & (df.subnormal_LDA == 1), ['rm', 'nome_rm', 'numero_de_domicilios', 'numero_moradores']].groupby('rm').agg({
    'nome_rm': 'first',
    'rm': 'count',
    'numero_de_domicilios': 'sum',
    'numero_moradores': 'sum'
})

,nome_rm,rm,numero_de_domicilios,numero_moradores
rm,,,,
13,Rio de Janeiro,1237,258660.0,927628.0
14,São Paulo,1680,359322.0,1416577.0
15,Baixada Santista,164,44777.0,172694.0
16,Campinas,224,39479.0,155384.0


In [13]:
# Resultados Parciais espacializados para as RMS ([13, 14, 15, 16])
gdf_setores = gpd.read_file('data/censo_2000/br_set_cens_2000_4326.zip')

In [14]:
# df.dtypes


In [15]:
gdf_setores.loc[:, 'Cod_setor'] = gdf_setores.CD_GEOCODI.astype('int')

In [16]:
gdf_setores_joined = gdf_setores.set_index('Cod_setor').join(df.set_index('Cod_setor'), how='left', lsuffix='_df')

In [17]:
gdf_setores_joined.loc[:, 'legenda'] = 'Setores Comuns'
gdf_setores_joined.loc[gdf_setores_joined.subnormal_LDA.isna(), 'legenda'] = 'Sem Informação'

In [18]:
subnormal = (gdf_setores_joined.tipo_setor == 1)

In [19]:
gdf_setores_joined.loc[subnormal, 'legenda'] = 'Subnormais'

In [20]:
precarios = (gdf_setores_joined.subnormal_LDA == 1.0)

In [21]:
gdf_setores_joined.loc[(precarios & ~subnormal), 'legenda'] = 'Precários'

In [22]:
# gdf_setores

In [23]:
gdf_setores_joined.legenda.value_counts()

legenda
Setores Comuns    57361
Sem Informação    57068
Subnormais         7783
Precários          5800
Name: count, dtype: int64

In [24]:
# gdf_setores_joined.loc[~gdf_setores_joined.subnormal_LDA.isna(), ['geometry', 'subnormal_LDA', 'legenda']].to_file('resultados/test_rms_samples_X_2.gpkg', driver='GPKG')

In [ ]:
# result

In [ ]:
df.subnormal_LDA.value_counts()

In [ ]:
df.subnormal_RandomForest.value_counts()

In [ ]:
df.columns

In [ ]:
r_ufs = df[df.rm.isin(cidades_fora_rms)].loc[(subnormal), ['nome_UF', 'subnormal']].groupby('nome_UF').count().reset_index()

In [ ]:
r_ufs

In [ ]:
df_setores = df[df.rm.isin(cidades_fora_rms)].loc[:, ['nome_UF', 'subnormal', 'Cod_setor']].groupby('nome_UF').agg({'subnormal': 'sum', 'Cod_setor':'count'})

In [ ]:
df_setores[df_setores.subnormal < 20]

In [ ]:
df_setores[df_setores.subnormal < 20].sum()

In [ ]:
cidades_fora_rms

In [ ]:
r_ufs[r_ufs.subnormal >= 20]

In [ ]:
r_ufs[r_ufs.subnormal < 20]

In [ ]:
# df[(df.rm.isin(cidades_fora_rms))]

In [ ]:
# Processa cada uma das UFs
df_uf = df[df.rm.isin(cidades_fora_rms)]
ufs = r_ufs[r_ufs.subnormal >= 20].nome_UF.to_list()

for uf in ufs:

    uf_filter = (df.nome_UF == uf) & (df.rm.isin(cidades_fora_rms))
    print(uf)

    registro = processa_modelos(uf_filter, uf=uf)
    # Salva as predicoes em atributos
    result.append(registro)


In [ ]:
gr_filter = (df.cod_grande_regiao == 4.) & (df.subnormal_LDA.isna())

In [ ]:
## TODO
## Processar as cidades restantes por MacroRegião 
## Região 4 (Sul) não possui licoes suficientes de municípios com setores subnormais
for gr in [1., 2., 3., 5.]:
    gr_filter = (df.cod_grande_regiao == gr) & (~df.processado)
    print(gr)

    if len(df[gr_filter]) > 0:
        registro = processa_modelos(gr_filter, uf=f'Macro Região {str(gr)}')
        # Salva as predicoes em atributos
        result.append(registro)

In [ ]:
df[df.cod_grande_regiao == 4.]

In [ ]:
pd.DataFrame(result).to_excel('resultados/geral_por_RM_UF.xlsx')

In [ ]:
pd.DataFrame(result).quantidade_de_municipios_processados.sum()

In [ ]:
df.subnormal_LDA.value_counts()

In [ ]:
df.subnormal_RandomForest.value_counts()
# df.columns

In [ ]:
df.subnormal.value_counts()

In [ ]:
df.to_csv('resultados/modelo_preditivo.csv.zip', compression='zip')

In [ ]:
df[df.subnormal_LDA.isna()]

In [ ]:
df

In [ ]:
df.processado.value_counts()